In [8]:
import sys
from pyspark import SparkContext, SparkConf
import time
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, BooleanType


In [ ]:
#### Driver program

# start spark with 1 worker thread
sc = SparkContext("local[8]")
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)

print("OK")

In [12]:
### combine task_event and task_usage and compare


# Task Events Schema definieren
task_events_schema = StructType([
    StructField("time", IntegerType(), True),
    StructField("missing_info", IntegerType(), True),
    StructField("job_id", IntegerType(), False),
    StructField("task_index", IntegerType(), False),
    StructField("machine_id", IntegerType(), True),
    StructField("event_type", IntegerType(), False),
    StructField("user", StringType(), True),
    StructField("scheduling_class", IntegerType(), True),
    StructField("priority", IntegerType(), False),
    StructField("cpu_request", FloatType(), True),
    StructField("memory_request", FloatType(), True),
    StructField("disk_space_request", FloatType(), True),
    StructField("different_machines_restriction", BooleanType(), True)
])

# Task Usage Schema definieren
task_usage_schema = StructType([
    StructField("start_time", IntegerType(), False),
    StructField("end_time", IntegerType(), False),
    StructField("job_id", IntegerType(), False),
    StructField("task_index", IntegerType(), False),
    StructField("machine_id", IntegerType(), False),
    StructField("cpu_rate", FloatType(), True),
    StructField("canonical_memory_usage", FloatType(), True),
    StructField("assigned_memory_usage", FloatType(), True),
    StructField("unmapped_page_cache", FloatType(), True),
    StructField("total_page_cache", FloatType(), True),
    StructField("maximum_memory_usage", FloatType(), True),
    StructField("disk_io_time", FloatType(), True),
    StructField("local_disk_space_usage", FloatType(), True),
    # ... weitere Felder
])




task_events_paths = [
    "google-dataset/task_events/part-00265-of-00500.csv.gz",
    "google-dataset/task_events/part-00266-of-00500.csv.gz",
    "google-dataset/task_events/part-00267-of-00500.csv.gz",
    "google-dataset/task_events/part-00268-of-00500.csv.gz",
    "google-dataset/task_events/part-00269-of-00500.csv.gz",
]

task_usage_paths = [
    "google-dataset/task_usage/part-00265-of-00500.csv.gz",
    "google-dataset/task_usage/part-00266-of-00500.csv.gz",
    "google-dataset/task_usage/part-00267-of-00500.csv.gz",
    "google-dataset/task_usage/part-00268-of-00500.csv.gz",
    "google-dataset/task_usage/part-00269-of-00500.csv.gz",
]



# load data
df_events = sqlContext.read.csv("google-dataset/task_events/part-*-of-*.csv.gz", 
                           schema=task_events_schema, 
                           header=False)

df_usage = sqlContext.read.csv("google-dataset/task_usage/part-*-of-*.csv.gz",
                          schema=task_usage_schema,
                          header=False)


# only keep valid data remove NULL
df_events_clean = df_events.filter(
    (df_events.cpu_request.isNotNull()) |
    (df_events.memory_request.isNotNull()) |
    (df_events.disk_space_request.isNotNull())
)

df_usage_clean = df_usage.filter(
    (df_usage.cpu_rate.isNotNull()) |
    (df_usage.canonical_memory_usage.isNotNull()) |
    (df_usage.local_disk_space_usage.isNotNull())
)


print("task_events")
df_events_clean.show()

print("task_usage")
df_usage_clean.show()


task_events
+----+------------+---------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|time|missing_info|   job_id|task_index|machine_id|event_type|                user|scheduling_class|priority|cpu_request|memory_request|disk_space_request|different_machines_restriction|
+----+------------+---------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|NULL|        NULL|     NULL|        70|  56896039|         1|XXdY557FQk791swgp...|               2|       4|     0.0625|        0.0636|          5.817E-5|                          NULL|
|NULL|        NULL|     NULL|       256|2107285354|         1|XXdY557FQk791swgp...|               2|       4|     0.0625|        0.0636|          5.817E-5|                          NULL|
|NULL|        NULL|515042969|         5| 351621284|  

In [13]:
### COMBINE THE TWO TABLES

from pyspark.sql.functions import col
from pyspark.sql.functions import avg, max, sum


# Join job_id, task_index
df_combined = df_events_clean.join(
    df_usage_clean,
    on=["job_id", "task_index"],
    how="inner"
)



# aggregate data: use average and max
df_aggregated = df_combined.groupBy("job_id", "task_index", 
                                    "cpu_request", 
                                    "memory_request", 
                                    "disk_space_request").agg(
    avg("cpu_rate").alias("avg_cpu_usage"),
    max("cpu_rate").alias("max_cpu_usage"),
    avg("canonical_memory_usage").alias("avg_memory_usage"),
    max("maximum_memory_usage").alias("max_memory_usage"),
    avg("local_disk_space_usage").alias("avg_disk_usage"),
    max("local_disk_space_usage").alias("max_disk_usage")
)


df_aggregated.show()

+----------+----------+-----------+--------------+------------------+--------------------+-------------+--------------------+----------------+--------------------+--------------+
|    job_id|task_index|cpu_request|memory_request|disk_space_request|       avg_cpu_usage|max_cpu_usage|    avg_memory_usage|max_memory_usage|      avg_disk_usage|max_disk_usage|
+----------+----------+-----------+--------------+------------------+--------------------+-------------+--------------------+----------------+--------------------+--------------+
|1836173247|        31|   0.004982|       0.01202|          1.831E-4|1.816674719805862...|      5.15E-4|0.003858631320370653|         0.01186|4.893868136408066E-5|      1.621E-4|
|1412625411|        41|     0.0625|       0.01746|          0.003395|                 0.0|          0.0|                 0.0|             0.0|                 0.0|           0.0|
| 501114088|         8|    0.03125|      0.004349|          3.815E-6|4.182235113167986...|      0.01501|4

In [19]:
### CORRELATION ANALYSIS

from pyspark.sql.functions import corr, when, col, count



# caclulate correlations
correlations = df_aggregated.select(
    corr("cpu_request", "avg_cpu_usage").alias("cpu_request_vs_avg_usage_corr"),
    corr("cpu_request", "max_cpu_usage").alias("cpu_request_vs_max_usage_corr"),
    corr("memory_request", "avg_memory_usage").alias("mem_request_vs_avg_usage_corr"),
    corr("memory_request", "max_memory_usage").alias("mem_request_vs_max_usage_corr"),
    corr("disk_space_request", "avg_disk_usage").alias("disk_request_vs_avg_usage_corr"),
    corr("disk_space_request", "max_disk_usage").alias("disk_request_vs_max_usage_corr")
)

print("correlations")
correlations.show()


# cou category
df_categorized = df_aggregated.withColumn(
    "cpu_request_category",
    when(col("cpu_request") < 0.1, "low")
    .when((col("cpu_request") >= 0.1) & (col("cpu_request") < 0.5), "medium")
    .otherwise("high")
)

# Memory category
df_categorized = df_categorized.withColumn(
    "memory_request_category",
    when(col("memory_request") < 0.1, "low")
    .when((col("memory_request") >= 0.1) & (col("memory_request") < 0.5), "medium")
    .otherwise("high")
)


# Statistics per category
cpu_category_stats = df_categorized.groupBy("cpu_request_category").agg(
    avg("cpu_request").alias("avg_cpu_request"),
    avg("avg_cpu_usage").alias("avg_cpu_consumption"),
    avg("max_cpu_usage").alias("avg_max_cpu_consumption"),
    count("*").alias("task_count")
).orderBy("cpu_request_category")

print("cpu_category_stats")
cpu_category_stats.show()

# Durchschnittlicher Verbrauch pro Memory-Anforderungskategorie
memory_category_stats = df_categorized.groupBy("memory_request_category").agg(
    avg("memory_request").alias("avg_memory_request"),
    avg("avg_memory_usage").alias("avg_memory_consumption"),
    avg("max_memory_usage").alias("avg_max_memory_consumption"),
    count("*").alias("task_count")
).orderBy("memory_request_category")

print("memory_category_stats")
memory_category_stats.show()



correlations
+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------------------------+------------------------------+
|cpu_request_vs_avg_usage_corr|cpu_request_vs_max_usage_corr|mem_request_vs_avg_usage_corr|mem_request_vs_max_usage_corr|disk_request_vs_avg_usage_corr|disk_request_vs_max_usage_corr|
+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------------------------+------------------------------+
|           0.1572320058457296|          0.06685235959595083|           0.6488625317615484|            0.658854190821016|           -0.1514062269039782|          -0.20758054836462286|
+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------------------------+------------------------------+

cpu_category_stats
+--------------------+--------------------+----

In [22]:
### efficiency stats

from pyspark.sql.functions import col, expr

df_efficiency = df_aggregated.withColumn(
    "cpu_utilization_ratio", 
    col("avg_cpu_usage") / col("cpu_request")
).withColumn(
    "memory_utilization_ratio",
    col("avg_memory_usage") / col("memory_request")
).withColumn(
    "disk_utilization_ratio",
    col("avg_disk_usage") / col("disk_space_request")
)

# Statistiken über Utilization Ratios
efficiency_stats = df_efficiency.select(
    avg("cpu_utilization_ratio").alias("avg_cpu_util_ratio"),
    avg("memory_utilization_ratio").alias("avg_mem_util_ratio"),
    avg("disk_utilization_ratio").alias("avg_disk_util_ratio")
)

print("efficiency_stats")
efficiency_stats.show()



# Perzentile für CPU
cpu_percentiles = df_aggregated.selectExpr(
    "percentile(cpu_request, array(0.25, 0.5, 0.75, 0.9, 0.95)) as cpu_request_percentiles",
    "percentile(avg_cpu_usage, array(0.25, 0.5, 0.75, 0.9, 0.95)) as cpu_usage_percentiles"
)

print("cpu_percentiles")
cpu_percentiles.show(truncate=False)



efficiency_stats
+-------------------+-------------------+-------------------+
| avg_cpu_util_ratio| avg_mem_util_ratio|avg_disk_util_ratio|
+-------------------+-------------------+-------------------+
|0.09022841037270246|0.12263786135522889|0.09161409969161441|
+-------------------+-------------------+-------------------+

cpu_percentiles
+-------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|cpu_request_percentiles                                                        |cpu_usage_percentiles                                                        |
+-------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|[0.02812250005081296, 0.0625, 0.0625, 0.11559999734163284, 0.11559999734163284]|[0.0, 0.0, 4.295284771139268E-4, 0.0033412727148970593, 0.010346246170558748]|
+---------------